작업하시기 편하게 BaseLine만들어놨습니다.
- 모델링 돌릴때, 각자 상황에 맞게 전처리를 바꾸시기도 하고, 
- 여유가 되면 다른모델을 시도해봐도  좋습니다.
- 즐거운 하루 보내세요

# Module Import 

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob

from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


# Data Load

In [3]:
ifood_df = pd.read_csv('../ifood_df.csv')
project_df = pd.read_csv('../ml_project1_data.csv')

labelencoder

In [4]:
project_df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


In [6]:
project_df.shape

(2240, 29)

# Preprocessing

In [8]:
## 필요없는 변수 제거
project_df = project_df.drop(['Z_CostContact', 'Z_Revenue'], axis=1)

## 'Income'결측치 제거와 정규화 
project_df = project_df[project_df['Income'].notna()].reset_index(drop=True)
project_df['Income']=StandardScaler().fit_transform(project_df['Income'].values.reshape(-1,1))

## 'Year_Birth'2023년으로 기준 나이 -> 고객등록일 기준으로 바꿀필요 있음!!
project_df['Year_Birth'] = project_df['Year_Birth'].apply(lambda x : np.abs(x-2023+1))

## 'Year_Birth' : normalization
project_df['Year_Birth']=StandardScaler().fit_transform(project_df['Year_Birth'].values.reshape(-1,1))

## 'Accept_all'변수 추가
accept_li = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp4', 'Response']
project_df['Accept_all'] = project_df[accept_li].sum(axis=1)

num_li = ['NumDealsPurchases', 'NumWebPurchases','NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']
Mnt_li = ['MntWines', 'MntFruits','MntMeatProducts', 'MntFishProducts', 'MntSweetProducts','MntGoldProds']

## education 범주화
def make_education_column(x:str):
    if x == '2n Cycle':
        x = 0
    elif x == 'Basic':
        x = 1
    elif x == 'Graduation':
        x = 2
    elif x == 'Master':
        x = 3
    else :
        x = 4
    return x
project_df['Education'] = project_df['Education'].apply(lambda x : make_education_column(x))

## 'marital_status' 범주화 
dic_marital = {}
for ind, status in enumerate(set(project_df['Marital_Status'])) :
    dic_marital[status] = ind
project_df['Marital_Status'] = project_df['Marital_Status'].replace(dic_marital)

## 'Recency' 정규화 
project_df['Recency'] = StandardScaler().fit_transform(project_df['Recency'].values.reshape(-1,1))

## 'year', 'month', 'day' 변수 추가
project_df['year'] = project_df['Dt_Customer'].str.split('-').str.get(0).astype(int)
project_df['month'] = project_df['Dt_Customer'].str.split('-').str.get(1).astype(int)
project_df['day'] = project_df['Dt_Customer'].str.split('-').str.get(2).astype(int)

## 'Mnt_all'변수 추가 및 모든 Mnt변수들 정규화 
project_df['Mnt_all'] = project_df[Mnt_li].sum(axis=1)
mnt_every_li = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'Mnt_all']

scaler = StandardScaler()            
scaler.fit(project_df[mnt_every_li])                  
df_s = scaler.transform(project_df[mnt_every_li])     
df_s = pd.DataFrame(df_s, columns = mnt_every_li)
project_df[mnt_every_li] = df_s

## 'Num_all'변수 추가 및 모든 Num변수 정규화 
project_df['Num_all'] = project_df[num_li].sum(axis=1)
num_every_li = ['NumDealsPurchases', 'NumWebPurchases','NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Num_all'] 

scaler = StandardScaler() 
scaler.fit(project_df[num_every_li])                  
df_s = scaler.transform(project_df[num_every_li])     
df_s = pd.DataFrame(df_s, columns = num_every_li)
project_df[num_every_li] = df_s

project_df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp1,AcceptedCmp2,Complain,Response,Accept_all,year,month,day,Mnt_all,Num_all
0,5524,0.986443,2,7,0.234063,0,0,2012-09-04,0.310532,0.978226,...,0,0,0,1,1,2012,9,4,1.675488,1.620547
1,2174,1.236801,2,7,-0.234559,1,1,2014-03-08,-0.380509,-0.872024,...,0,0,0,0,0,2014,3,8,-0.962358,-1.263455
2,4141,0.318822,2,0,0.769478,0,0,2013-08-21,-0.795134,0.358511,...,0,0,0,0,0,2013,8,21,0.280250,0.659213
3,6182,-1.266777,2,0,-1.017239,1,0,2014-02-10,-0.795134,-0.872024,...,0,0,0,0,0,2014,2,10,-0.919224,-0.851455
4,5324,-1.016420,4,1,0.240221,1,0,2014-01-19,1.554407,-0.391671,...,0,0,0,0,0,2014,1,19,-0.307044,0.521879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2211,10870,0.151917,2,1,0.356642,0,1,2013-06-13,-0.104093,1.197646,...,0,0,0,0,0,2013,6,13,1.217598,0.384546
2212,4001,1.904422,4,0,0.467539,2,1,2014-06-10,0.241428,0.299208,...,1,0,0,0,1,2014,6,10,-0.270546,1.208546
2213,7270,-1.016420,2,2,0.188091,0,0,2014-01-25,1.450751,1.787710,...,0,0,0,0,2,2014,1,25,1.051696,0.659213
2214,8235,1.069896,3,0,0.675388,0,1,2014-01-24,-1.417072,0.364441,...,0,0,0,0,0,2014,1,24,0.391404,0.796546


# Feature Selection

In [28]:
features = ['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 
 'Teenhome', 'Recency', 'MntWines', 'MntFruits',
 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
 'AcceptedCmp2', 'Complain', 'Accept_all', 'year', 'month',
 'day', 'Num_all']

classification_target = ['Response']
regression_target = ['Mnt_all']

# Data Split

In [22]:
project_df[features]

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Accept_all,year,month,day,Num_all
0,0.986443,2,7,0.234063,0,0,0.310532,0.978226,1.549429,1.690227,...,0,0,0,0,0,1,2012,9,4,1.620547
1,1.236801,2,7,-0.234559,1,1,-0.380509,-0.872024,-0.637328,-0.717986,...,0,0,0,0,0,0,2014,3,8,-1.263455
2,0.318822,2,0,0.769478,0,0,-0.795134,0.358511,0.569159,-0.178368,...,0,0,0,0,0,0,2013,8,21,0.659213
3,-1.266777,2,0,-1.017239,1,0,-0.795134,-0.872024,-0.561922,-0.655551,...,0,0,0,0,0,0,2014,2,10,-0.851455
4,-1.016420,4,1,0.240221,1,0,1.554407,-0.391671,0.418348,-0.218505,...,0,0,0,0,0,0,2014,1,19,0.521879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2211,0.151917,2,1,0.356642,0,1,-0.104093,1.197646,0.418348,0.066913,...,0,0,0,0,0,0,2013,6,13,0.384546
2212,1.904422,4,0,0.467539,2,1,0.241428,0.299208,-0.662463,-0.610954,...,0,0,1,0,0,1,2014,6,10,1.208546
2213,-1.016420,2,2,0.188091,0,0,1.450751,1.787710,0.544024,0.223001,...,1,0,0,0,0,2,2014,1,25,0.659213
2214,1.069896,3,0,0.675388,0,1,-1.417072,0.364441,0.091591,0.209622,...,0,0,0,0,0,0,2014,1,24,0.796546


## Classification Split & Modeling

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier

In [23]:
X_train, X_test, y_train, y_test = train_test_split(project_df[features], project_df[classification_target],
                                                    test_size=0.2, random_state=42,
                                                    stratify = project_df[classification_target])


clf = RandomForestClassifier(max_depth=2, random_state=42)
clf.fit(X_train, y_train)
clf.predict(X_test)

C:\Users\jihoo\AppData\Local\Temp\ipykernel_21520\801986344.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

여기에 
- 10 fold로 모델링 해주시고, 
- metric은 f1-score로 평가해주시고, 
- feature importance뽑아주세요. 
- 변수들 heatmap그려주세요

## Regression Split & Modeling

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor

In [33]:
X_train, X_test, y_train, y_test = train_test_split(project_df[features], project_df[regression_target],
                                                    test_size=0.2, random_state=42,
                                                    stratify = project_df[classification_target])

regr = RandomForestRegressor(max_depth=2, random_state=42)
regr.fit(X_train, y_train)
regr.predict(X_test)

C:\Users\jihoo\AppData\Local\Temp\ipykernel_21520\3772058761.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X_train, y_train)


array([ 1.21472660e+00, -4.30886087e-01, -8.14188709e-01, -7.56899643e-02,
        5.77662329e-01, -8.14188709e-01, -8.14188709e-01, -8.14188709e-01,
       -8.14188709e-01, -7.12363805e-01,  7.29838789e-01, -7.12363805e-01,
       -8.14188709e-01, -6.97122164e-01,  2.22061951e-01, -8.14188709e-01,
       -8.14188709e-01, -4.08314800e-02, -8.14188709e-01, -8.14188709e-01,
        2.02457698e-01, -7.65602650e-01,  6.67308426e-01, -8.14188709e-01,
       -6.51768237e-02,  1.06678468e+00,  7.19232329e-01,  4.50499548e-01,
       -8.14188709e-01,  7.45538138e-01, -8.14188709e-01,  2.02457698e-01,
        1.40492441e+00, -8.14188709e-01,  6.73677370e-01,  1.40492441e+00,
        1.09149099e+00,  1.09149099e+00, -8.14188709e-01,  1.00544816e+00,
        5.46544570e-01, -7.04423844e-01,  5.70910179e-01, -8.14188709e-01,
        3.50216584e-01, -8.14188709e-01, -8.14188709e-01, -8.14188709e-01,
       -7.86034150e-01,  1.61420286e+00, -7.78094189e-01, -8.14188709e-01,
       -8.14188709e-01, -

여기에 
- 10 fold로 모델링 해주시고,
- metric은 MSE로 평가해주시고, 
- feature importance뽑아주세요. 
- 변수들 heatmap그려주세요